In [24]:
from tree_sitter import Language, Parser, remove_comments_and_docstrings

ImportError: cannot import name 'remove_comments_and_docstrings'

In [16]:
import pickle
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig
import torch.nn as nn
import json
import random
import argparse
import multiprocessing
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm
import os
import logging
from tree_sitter import Language, Parser
import networkx as nx
import numpy as np
import sys
sys.setrecursionlimit(5000)

In [18]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
Language.build_library(
    'build/my-language.so',
    [
        '/data/code/tree-sitter/tree-sitter-ruby',
        '/data/code/tree-sitter/tree-sitter-javascript',
        '/data/code/tree-sitter/tree-sitter-go',
        '/data/code/tree-sitter/tree-sitter-python',
        '/data/code/tree-sitter/tree-sitter-java',
        '/data/code/tree-sitter/tree-sitter-php',
    ]
)
language = Language('build/my-language.so', 'java')
parser = Parser()
parser.set_language(language)

In [10]:
parser

In [11]:
src = 'public class Hello{\nprivate String text = "Hello World!";\npublic void print(int value) {\nif(value>100) System.out.println(value);}}'

In [20]:
def get_ast_nx(example, parser, lang):
    new_code = remove_comments_and_docstrings(example.raw_code, lang)
    print("new_code:",new_code)
    tree = parser.parse(bytes(new_code, 'utf-8'))
    G = nx.Graph()
    cursor = tree.walk()
    traverse(cursor, G, came_up=False, node_tag=0, node_sum=0, parent_dict={})
    return Example(
        idx=example.idx,
        source=new_code,
        target=example.target,
        ast=G
    )

In [21]:
def get_ast_and_token(examples, parser, lang):
    ast_list = []
    tokens_list = []
    logger.info("Parse AST trees and obtain leaf tokens")
    i = 0
    j = 0
    leaves_list=[]
    for example in tqdm(examples):
        ast_example = get_ast_nx(example, parser, lang)
        G = ast_example.ast
        ast_list.append(G)
        T = nx.dfs_tree(G, 0)
        leaves = [x for x in T.nodes() if T.out_degree(x) ==
                  0 and T.in_degree(x) == 1]
        leaves_list.append(leaves)
        tokens_dict = {}
        for leaf in leaves:
            feature = G.nodes[leaf]['features']
            if feature.type != 'comment':
                start = feature.start_point
                end = feature.end_point
                token = index_to_code_token([start, end], ast_example.source)
                if i == 0:
                    print('leaf: ', leaf, 'start: ', start,
                          ', end: ', end, ', token: ', token)
                tokens_dict[leaf] = token

        if i == 0:
            print(T.nodes)
            print(T.edges)
            print('raw_code: ', ast_example.source)
            print('leaves: ', leaves)
            i += 1

        tokens_list.append(tokens_dict)
        j+=1
    print('example length:',j)
    print('ast list length', len(ast_list))
    print('tokens list length', len(tokens_list))
    print('tokens 0: ')
    print(tokens_list[0])
    return ast_list, tokens_list , leaves_list

In [22]:
ast_list, tokens_list, leaves = get_ast_and_token(src, parser, 'java')

06/04/2022 04:33:19 - INFO - __main__ -   Parse AST trees and obtain leaf tokens
  0%|          | 0/131 [00:00<?, ?it/s]


NameError: name 'remove_comments_and_docstrings' is not defined

In [23]:
src_clean = remove_comments_and_docstrings(src, 'java')

NameError: name 'remove_comments_and_docstrings' is not defined